In [40]:
import openai, sys
print("openai version:", openai.__version__)
print("python version:", sys.version)

openai version: 1.108.0
python version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]


In [41]:
import os
os.environ["OPENAI_API_KEY"] = "sk..."

assert os.environ.get("OPENAI_API_KEY","").startswith("sk-"), "API key missing or malformed."
print("API key set ✔")


API key set ✔


In [42]:
from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

resp = client.responses.create(
    model="gpt-4o-mini",
    input=[{"role": "user", "content": "Say hello in exactly 7 words."}],
    max_output_tokens=30,
)
print(resp.output_text)

Hello! I hope you're having a great day!


In [43]:
def extract_first_json_object(s: str) -> str:
    """
    Returns the first top-level JSON object found in s.
    Raises ValueError if none or braces unbalanced.
    """
    start = s.find("{")
    if start == -1:
        raise ValueError("No '{' found in model output.")
    depth = 0
    in_str = False
    esc = False
    for i in range(start, len(s)):
        c = s[i]
        if in_str:
            if esc:
                esc = False
            elif c == "\\":
                esc = True
            elif c == '"':
                in_str = False
        else:
            if c == '"':
                in_str = True
            elif c == "{":
                depth += 1
            elif c == "}":
                depth -= 1
                if depth == 0:
                    return s[start:i+1]
    raise ValueError("Unbalanced JSON braces in model output.")


In [44]:
EVAL_USER_TEMPLATE = """\
Task: {task_type}

Input features (optional): {input_brief}

Model's EXPLANATION for its prediction (XAI):
\"\"\"{xai_explanation}\"\"\"

Objective (Simulatability):
1) Guess the model's LABEL from the explanation (use only the valid label set).
2) Give confidence 0–1.
3) Short rationale.

Subjective (1–5): clarity, helpfulness, trust, fairness, confidence_after.
Also: one short free-text comment.

Binary flags: suspected_proxy_bias, overfitting_smell, leakage_risk.

Return JSON only with the fields we ask for.
"""

# Example inputs — replace with your real task & explanation
task_type = "sentiment classification"
valid_labels = ["POSITIVE","NEGATIVE"]
input_brief = "Short hotel review about cleanliness and service."
xai_explanation = (
    "The model focused on 'spotless room', 'friendly staff', and 'would definitely return', "
    "pushing toward POSITIVE. 'small lobby' had low weight."
)

user_prompt = EVAL_USER_TEMPLATE.format(
    task_type=f"{task_type} (valid labels: {', '.join(valid_labels)})",
    input_brief=input_brief,
    xai_explanation=xai_explanation
)

print(user_prompt[:400] + " ...")

Task: sentiment classification (valid labels: POSITIVE, NEGATIVE)

Input features (optional): Short hotel review about cleanliness and service.

Model's EXPLANATION for its prediction (XAI): 
"""The model focused on 'spotless room', 'friendly staff', and 'would definitely return', pushing toward POSITIVE. 'small lobby' had low weight."""

Objective (Simulatability):
1) Guess the model's LABEL from ...


In [48]:
import json
from copy import deepcopy
from jsonschema import validate, ValidationError

# === Our schema (same as before) ===
OUTPUT_SCHEMA = {
    "name": "xai_eval_result",
    "schema": {
        "type": "object",
        "properties": {
            "persona_id": {"type": "string"},
            "objective": {
                "type": "object",
                "properties": {
                    "simulated_label": {"type": "string"},
                    "confidence_0to1": {"type": "number"},
                    "rationale": {"type": "string"}
                },
                "required": ["simulated_label", "confidence_0to1", "rationale"],
                "additionalProperties": False
            },
            "subjective": {
                "type": "object",
                "properties": {
                    "clarity_1to5": {"type": "integer"},
                    "helpfulness_1to5": {"type": "integer"},
                    "trust_1to5": {"type": "integer"},
                    "fairness_1to5": {"type": "integer"},
                    "confidence_after_1to5": {"type": "integer"},
                    "free_text_feedback": {"type": "string"}
                },
                "required": [
                    "clarity_1to5","helpfulness_1to5","trust_1to5",
                    "fairness_1to5","confidence_after_1to5","free_text_feedback"
                ],
                "additionalProperties": False
            },
            "flags": {
                "type": "object",
                "properties": {
                    "suspected_proxy_bias": {"type": "boolean"},
                    "overfitting_smell": {"type": "boolean"},
                    "leakage_risk": {"type": "boolean"}
                },
                "required": ["suspected_proxy_bias","overfitting_smell","leakage_risk"],
                "additionalProperties": False
            }
        },
        "required": ["persona_id","objective","subjective","flags"],
        "additionalProperties": False
    },
    "strict": True
}

JSON_SCHEMA_INNER = OUTPUT_SCHEMA["schema"]

# === A concrete JSON *template* the model can mimic exactly ===
JSON_TEMPLATE_EXAMPLE = {
    "persona_id": "SAMPLE_PERSONA_ID",
    "objective": {
        "simulated_label": "POSITIVE",
        "confidence_0to1": 0.85,
        "rationale": "Brief reason here."
    },
    "subjective": {
        "clarity_1to5": 4,
        "helpfulness_1to5": 4,
        "trust_1to5": 3,
        "fairness_1to5": 4,
        "confidence_after_1to5": 4,
        "free_text_feedback": "One short sentence of feedback."
    },
    "flags": {
        "suspected_proxy_bias": False,
        "overfitting_smell": False,
        "leakage_risk": False
    }
}

def coerce_common_near_misses(d: dict) -> dict:
    """
    Maps common 'near-miss' outputs into our schema shape.
    Handles cases like:
      LABEL -> objective.simulated_label
      confidence -> objective.confidence_0to1
      rationale -> objective.rationale
      comment -> subjective.free_text_feedback
      subjective keys without _1to5 suffix
      flags at top-level, etc.
    """
    # If already valid-ish, return quickly
    if isinstance(d, dict) and "objective" in d and "subjective" in d and "flags" in d:
        return d

    fixed = {
        "objective": {"simulated_label": None, "confidence_0to1": None, "rationale": None},
        "subjective": {
            "clarity_1to5": None,
            "helpfulness_1to5": None,
            "trust_1to5": None,
            "fairness_1to5": None,
            "confidence_after_1to5": None,
            "free_text_feedback": ""
        },
        "flags": {
            "suspected_proxy_bias": False,
            "overfitting_smell": False,
            "leakage_risk": False
        }
    }

    # Flattened top-level near-misses
    if "LABEL" in d:
        fixed["objective"]["simulated_label"] = d["LABEL"]
    if "simulated_label" in d:
        fixed["objective"]["simulated_label"] = d["simulated_label"]

    if "confidence" in d:
        fixed["objective"]["confidence_0to1"] = d["confidence"]
    if "confidence_0to1" in d:
        fixed["objective"]["confidence_0to1"] = d["confidence_0to1"]

    if "rationale" in d:
        fixed["objective"]["rationale"] = d["rationale"]

    # Subjective may be nested or flat
    subj = d.get("subjective", {})
    # Accept either with or without suffix; coerce to ints when possible
    def as_int(x):
        try: return int(x)
        except: return None

    for k_src, k_dst in [
        ("clarity", "clarity_1to5"),
        ("helpfulness", "helpfulness_1to5"),
        ("trust", "trust_1to5"),
        ("fairness", "fairness_1to5"),
        ("confidence_after", "confidence_after_1to5"),
    ]:
        if k_src in d: fixed["subjective"][k_dst] = as_int(d[k_src])
        if k_src in subj: fixed["subjective"][k_dst] = as_int(subj[k_src])
        if k_dst in d: fixed["subjective"][k_dst] = as_int(d[k_dst])
        if k_dst in subj: fixed["subjective"][k_dst] = as_int(subj[k_dst])

    # Free-text feedback sometimes called 'comment'
    if "comment" in d:
        fixed["subjective"]["free_text_feedback"] = str(d["comment"])
    if "free_text_feedback" in d:
        fixed["subjective"]["free_text_feedback"] = str(d["free_text_feedback"])
    if isinstance(subj, dict) and "free_text_feedback" in subj:
        fixed["subjective"]["free_text_feedback"] = str(subj["free_text_feedback"])

    # Flags show up top-level sometimes
    for k in ["suspected_proxy_bias", "overfitting_smell", "leakage_risk"]:
        if k in d:
            fixed["flags"][k] = bool(d[k])
    if "flags" in d and isinstance(d["flags"], dict):
        for k,v in d["flags"].items():
            if k in fixed["flags"]:
                fixed["flags"][k] = bool(v)

    return fixed

def eval_once(system_prompt, user_prompt, persona_id, temperature=0.6, model="gpt-4o-mini"):
    """
    One persona evaluates one explanation; asks the model for JSON matching our schema.
    1) First attempt: prompt with an explicit example template.
    2) Validate; if it fails, try to coerce common near-misses and re-validate.
    """
    # Include a tiny, concrete example so the model mirrors the exact keys.
    composed_user = (
        user_prompt
        + "\n\nIMPORTANT:\n"
          "- Return ONLY a valid JSON object.\n"
          "- Use EXACTLY these keys and nesting.\n"
          "- Example (fill with your own values):\n"
          + json.dumps(JSON_TEMPLATE_EXAMPLE, indent=2)
    )

    resp = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": composed_user},
        ],
        temperature=temperature,
        max_output_tokens=700,
    )

    text = resp.output_text.strip()
    raw_json = extract_first_json_object(text)
    data = json.loads(raw_json)

    # Add persona_id if missing so we can validate the full schema
    if "persona_id" not in data:
        data["persona_id"] = persona_id
    else:
        # Overwrite with our canonical id to keep it consistent
        data["persona_id"] = persona_id

    # First validation attempt
    try:
        validate(instance=data, schema=JSON_SCHEMA_INNER)
        return data
    except ValidationError:
        # Try to coerce near-misses into our schema
        repaired = coerce_common_near_misses(data)
        repaired["persona_id"] = persona_id
        validate(instance=repaired, schema=JSON_SCHEMA_INNER)  # will raise if still invalid
        return repaired

# ---- Try once ----
skeptical = "You are a risk-averse compliance auditor. You look for leakage, proxy bias, and overclaiming."
res1 = eval_once(skeptical, user_prompt, "skeptical_auditor")
res1


{'persona_id': 'skeptical_auditor',
 'objective': {'simulated_label': 'POSITIVE',
  'confidence_0to1': 0.9,
  'rationale': "The model identified strong positive indicators like 'spotless room' and 'friendly staff'."},
 'subjective': {'clarity_1to5': 5,
  'helpfulness_1to5': 5,
  'trust_1to5': 4,
  'fairness_1to5': 5,
  'confidence_after_1to5': 5,
  'free_text_feedback': 'The explanation clearly supports the positive label.'},
 'flags': {'suspected_proxy_bias': False,
  'overfitting_smell': False,
  'leakage_risk': False}}

In [53]:
PERSONAS = [
    ("skeptical_auditor",
     "You are a risk-averse compliance auditor. You look for leakage, proxy bias, and overclaiming."),
    ("busy_clinician",
     "You are a time-pressured clinician; value concise, actionable, trustworthy explanations that fit workflow."),
    ("fairness_advocate",
     "You are a fairness-focused advocate; scrutinize demographic harms and proxies."),
    ("enthusiastic_engineer",
     "You are a pragmatic ML engineer; emphasize faithfulness and implementation feasibility.")
]

def simulate_crowd(personas, user_prompt, runs_per_persona=3, temperature=0.7, model="gpt-4o-mini"):
    out = []
    for pid, sys in personas:
        for r in range(runs_per_persona):
            data = eval_once(
                system_prompt=sys,
                user_prompt=user_prompt,
                persona_id=pid,
                temperature=temperature,
                model=model,
            )
            data["meta"] = {"rep": r, "temperature": temperature, "model": model}
            out.append(data)
    return out

# Run the crowd simulation
crowd = simulate_crowd(PERSONAS, user_prompt, runs_per_persona=3)

# Print all results nicely
print(f"Total results: {len(crowd)}\n")
for i, res in enumerate(crowd, 1):
    print(f"=== Result {i} ({res['persona_id']}, rep {res['meta']['rep']}) ===")
    print(json.dumps(res, indent=2))
    print()



Total results: 12

=== Result 1 (skeptical_auditor, rep 0) ===
{
  "persona_id": "skeptical_auditor",
  "objective": {
    "simulated_label": "POSITIVE",
    "confidence_0to1": 0.9,
    "rationale": "The presence of key positive phrases strongly indicates a positive sentiment."
  },
  "subjective": {
    "clarity_1to5": 5,
    "helpfulness_1to5": 5,
    "trust_1to5": 4,
    "fairness_1to5": 4,
    "confidence_after_1to5": 5,
    "free_text_feedback": "The explanation was clear and well-supported."
  },
  "flags": {
    "suspected_proxy_bias": false,
    "overfitting_smell": false,
    "leakage_risk": false
  },
  "meta": {
    "rep": 0,
    "temperature": 0.7,
    "model": "gpt-4o-mini"
  }
}

=== Result 2 (skeptical_auditor, rep 1) ===
{
  "persona_id": "skeptical_auditor",
  "objective": {
    "simulated_label": "POSITIVE",
    "confidence_0to1": 0.9,
    "rationale": "The review highlights multiple positive aspects, indicating overall satisfaction."
  },
  "subjective": {
    "clari